In [0]:
import requests, json

base_url = "https://api.stackexchange.com/2.3/questions"
page = 1
while True:
    params = {
        "order": "desc",
        "sort": "creation",
        "site": "stackoverflow",
        "pagesize": 100,
        "page": page,
        "key": "rl_M1ZjooB6SNUAif5gAuSiS4Uzw" # TODO: store this in key vault later
    }
    resp = requests.get(base_url, params=params).json()
    if not resp.get("items"):
        break
    
    path = f"/mnt/so/bronze/questions_page={page}.json"
    dbutils.fs.put(path, json.dumps(resp), overwrite=True)
    
    if not ( resp.get("has_more", False) or resp.get("quota_remaining", 0) > 0 ):
        break
    page += 1


In [0]:
from pyspark.sql.functions import col, explode

df = spark.read.json("/mnt/so/bronze/*.json")

# Items array → rows
df_items = df.select(explode("items").alias("q"))

# Flatten minimal schema
df_flat = df_items.select(
    col("q.question_id").alias("id"),
    col("q.title"),
    col("q.view_count"),
    col("q.creation_date"),
    col("q.tags")
)

df_flat.write.format("delta").mode("overwrite").save("/mnt/so/silver/questions")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS stackoverflow_questions
USING DELTA
LOCATION '/mnt/so/silver/questions';


In [0]:
%sql
SELECT * FROM stackoverflow_questions ORDER BY id DESC LIMIT 10;

In [0]:
%sql
SELECT * FROM stackoverflow_questions ORDER BY id LIMIT 10;

In [0]:
%sql
SELECT COUNT(*) FROM stackoverflow_questions;

In [0]:
files = dbutils.fs.ls("/mnt/so/bronze/")
print(sum([f.size for f in files])/(1024**3), 'gb')

In [0]:
%sql
select dayish, count(dayish) c from (select int(creation_date/100000) dayish from stackoverflow_questions) x group by dayish;

In [0]:
from pyspark.sql.functions import avg
_sqldf.agg(avg('c')).collect()[0][0]
